In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt

# 設定中文字型為 SimSun
import matplotlib
# matplotlib.rcParams['font.sans-serif'] = ['SimSun']
# matplotlib.rcParams['axes.unicode_minus'] = False

# 本地資料夾位置
train_dir = "./all_data/train_resized"
test_dir = "./all_data/test_resized"

# 定義增強生成器和原始生成器
train_datagen = ImageDataGenerator(
    #rescale=1.0/255,
    rotation_range=20,  # 增強旋轉角度
    width_shift_range=0.3,
    horizontal_flip=True,
    brightness_range=[1.0, 1.75],  # 調整亮度範圍
    #channel_shift_range=55.0,  # 調整色調
    fill_mode='nearest',
    validation_split=0.2,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.2
)

# 創建原始數據生成器（只進行像素縮放）
original_datagen = ImageDataGenerator(validation_split=0.2)

print("增強生成器設定完成")


In [ ]:
tf.test.is_gpu_available()

In [ ]:
# 增強後訓練生成器
augmented_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    color_mode='grayscale',  # 灰階模式
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# 原始圖片的生成器
original_generator = original_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    color_mode='grayscale',  # 灰階模式
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# 將原始圖片和增強圖片結合
def combined_generator(original_gen, augmented_gen):
    while True:
        original_batch = original_gen.__next__()
        augmented_batch = augmented_gen.__next__()
        # 合併原始和增強圖片
        images = np.concatenate((original_batch[0], augmented_batch[0]), axis=0)
        labels = np.concatenate((original_batch[1], augmented_batch[1]), axis=0)
        yield images, labels

# 使用合併後的生成器進行訓練
train_dataset = combined_generator(original_generator, augmented_generator)

# 驗證圖縮放
validation_dataset = original_datagen.flow_from_directory(
    train_dir,
    target_size=(100, 100),
    color_mode='grayscale',  # 灰階模式
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    color_mode='grayscale',  # 灰階模式
    batch_size=32,
    class_mode='categorical',  # 確保標籤為 one-hot 格式
    shuffle=False  # 測試資料不需要打亂
)


In [ ]:
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout

from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D

batch_size = 16
steps_per_epoch = len(original_generator)
validation_steps = None

# 定義模型
def build_model(input_shape):
    class QuantizeLayer(layers.Layer):
        def call(self, inputs):
            # Scale to [0, 127] range
            min_val = tf.reduce_min(inputs)
            max_val = tf.reduce_max(inputs)
            scaled = (inputs - min_val) / (max_val - min_val) * 127.0
            return tf.clip_by_value(tf.round(scaled), 0.0, 127.0)
    
    model = Sequential([
        # 卷積層 1 + 激活函數 + 量化
        layers.Conv2D(4, (5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # QuantizeLayer(),

        # 卷積層 2 + 激活函數 + 量化
        layers.Conv2D(8, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # QuantizeLayer(),

        # 卷積層 3 + 激活函數 + 量化
        layers.Conv2D(8, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # QuantizeLayer(),

        # 卷積層 4 + 激活函數 + 量化
        layers.Conv2D(8, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # QuantizeLayer(),

        layers.GlobalAveragePooling2D(),
        Dense(16, activation='relu'),
        # QuantizeLayer(),
        Dense(5, activation='softmax')
    ])
    return model


# 指定輸入形狀
input_shape = (100, 100, 1)

# 建立模型
model = build_model(input_shape)

# 編譯模型
model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 配置回調函數
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=4,
    min_lr=1e-6
)

# model.summary()

# 模型訓練
history = model.fit(
    train_dataset,  # 訓練數據
    steps_per_epoch=steps_per_epoch,  # 每個 epoch 的步數
    epochs=20,
    # epochs=5,
    validation_data=validation_dataset,  # 驗證數據
    #callbacks=[early_stopping, reduce_lr],
)



In [ ]:
model.summary()

In [ ]:
model.save('model.h5')

In [ ]:
import os
import numpy as np
from keras.models import load_model

# 載入模型
model = load_model('model.h5')
# 定義輸出資料夾
output_dir = "model_parameters"
os.makedirs(output_dir, exist_ok=True)  # 確保輸出資料夾存在

# 導出模型參數
for idx, layer in enumerate(model.layers):
    # 確認該層是否有權重參數
    weights = layer.get_weights()
    if weights:  # 如果該層有參數（如卷積層、Dense層等）
        weights_file = os.path.join(output_dir, f"layer_{idx}_weights.dat")
        biases_file = os.path.join(output_dir, f"layer_{idx}_biases.dat")
        
        # 保存權重
        np.savetxt(weights_file, weights[0].flatten(), delimiter=',', fmt='%.6f')
        print(f"Saved weights for layer {idx} to {weights_file}")
        
        # 保存偏置
        if len(weights) > 1:  # 如果有偏置參數
            np.savetxt(biases_file, weights[1], delimiter=',', fmt='%.6f')
            print(f"Saved biases for layer {idx} to {biases_file}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# 設置字體為 DejaVu Sans
plt.rcParams['font.family'] = 'DejaVu Sans'

# 繪製損失曲線
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')
plt.savefig("loss_curve2.png")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.savefig("accuracy_curve2.png")
plt.show()

In [ ]:
train_dir = "./all_data/train_resized"
test_dir = "./all_data/test_resized"

class_names = ['A2C', 'A4C', 'OTHER', 'PLAX', 'PSAX']
# class_names = ['A2C', 'A4C', 'Other', 'PLAX', 'PSAX']

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100, 100),
    color_mode='grayscale',  # 灰階模式
    batch_size=32,
    class_mode='categorical',  # 確保標籤為 one-hot 格式
    classes=class_names,
    # classes=['A2C', 'A4C', 'Other', 'PLAX', 'PSAX'],
    shuffle=False  # 測試資料不需要打亂
)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import load_model

# 載入模型
model = load_model('model.h5')

# 獲取模型的預測結果
predictions = model.predict(test_dataset)

# 取得每張圖片的預測類別
predicted_classes = np.argmax(predictions, axis=1)

# 真實標籤
true_classes = test_dataset.classes

# 標籤名稱
class_labels = list(test_dataset.class_indices.keys())

import pandas as pd
results_df = pd.DataFrame({
    "File Name": test_dataset.filenames,
    "True Label": [class_labels[i] for i in true_classes],
    "Predicted Label": [class_labels[i] for i in predicted_classes]
})

# Calculate the number of correct predictions
correct_predictions = (results_df['True Label'] == results_df['Predicted Label']).sum()

# Calculate the total number of predictions
total_predictions = len(results_df)

# Calculate the proportion (accuracy)
accuracy = correct_predictions / total_predictions

top_2_predictions = np.argsort(predictions, axis=1)[:, -2:]

# Check if the true label is among the top-5 predictions
top_2_correct = [true_classes[i] in top_2_predictions[i] for i in range(len(true_classes))]

# Calculate Top-5 Accuracy
top_2_accuracy = np.mean(top_2_correct)

# Display the result
print(f"Number of correct predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Top-2 Accuracy: {top_2_accuracy:.2%}")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Calculate accuracy per class
class_accuracies = results_df.groupby('True Label').apply(
    lambda group: (group['True Label'] == group['Predicted Label']).mean()
)

# Convert the result to a DataFrame for plotting
class_accuracies_df = class_accuracies.reset_index()
class_accuracies_df.columns = ['Class', 'Accuracy']

# Plot the accuracy per class
plt.figure(figsize=(12, 6))
plt.bar(class_accuracies_df['Class'], class_accuracies_df['Accuracy'])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.title('Per-Class Accuracy')
plt.tight_layout()
plt.savefig("per_class_accuracy.png")
# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(results_df['True Label'], results_df['Predicted Label'], labels=results_df['True Label'].unique())

# Plot confusion matrix as a heatmap without annotations
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, cmap="Blues", cbar=True, xticklabels=results_df['True Label'].unique(), yticklabels=results_df['True Label'].unique(), annot=False)

# Add labels and title
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("confusion_matrix.png")
# Show the plot
plt.show()
